# Set up initial data paths

In [7]:
## Setup 
# import modules
import pandas as pd
import os
from datetime import datetime
import arcpy
import numpy as np
import urllib

# setup workspace folder
workspace = "F:/gis/PROJECTS/ResearchAnalysis/Monitoring/Data/Crash/"

# setup environment variables
arcpy.env.overwriteOutput = True
arcpy.env.workspace = "F:/GIS/PROJECTS/ResearchAnalysis/Monitoring/Data/Crash/CrashData/CrashData.gdb"

# create a spatial reference object for the output coordinate system 
# output projection for data going into SDE should be UTM Zone 10N (EPSG: 26910)
out_coordinate_system = arcpy.SpatialReference(26910)

# network path to connection files
filePath = "F:/GIS/DB_CONNECT"

# database file path 
sdeBase  = os.path.join(filePath, "Vector.sde")

# SDE feature classes needed for spatial joins
corridor = os.path.join(sdeBase, 'sde.SDE.Transportation\sde.SDE.Corridor')
trpa     = os.path.join(sdeBase, 'sde.SDE.Jurisdictions\sde.SDE.TRPA_bdy')

# define csv lat/long field names for xy table to point
x_coords = 'POINT_X'
y_coords = 'POINT_Y'

## Import raw data and sde highway collisions

In [8]:
# SDE feature class to update
crashSDE  = os.path.join(sdeBase, 'sde.SDE.Transportation\sde.SDE.Highway_Collisions')

# Get Crash Data
caCrashes = os.path.join(workspace, "BothCACounties_Crashes_1320_Unclean_v2.csv")
dfCACrash = pd.read_csv(caCrashes)

nvCrashes = os.path.join(workspace, "NV_LAKE TAHOE BASIN - ALL ROADS 2013-2020.csv")
dfNVCrash = pd.read_csv(nvCrashes)
    
# # in memory files
memory = "memory" + "\\"

C:\Users\amcclary\AppData\Local\Temp\ipykernel_32680\3716292085.py:6: DtypeWarning: Columns (6,17) have mixed types. Specify dtype option on import or set low_memory=False.
  dfCACrash = pd.read_csv(caCrashes)


## CA data transformation

In [ ]:

## CA data translation
# convert date/time and case info
#convert state/county/city, year/date/time
dfCACrash['State']       = "CA"
dfCACrash['City']        = dfCACrash['CITY']
dfCACrash['County']      = dfCACrash['COUNTY']
dfCACrash['Year']        = dfCACrash['ACCIDENT_YEAR']
dfCACrash['Date']        = dfCACrash['COLLISION_DATE']
dfCACrash['4DigTime']    = dfCACrash['COLLISION_TIME'].astype(str).str.zfill(4)
dfCACrash['Hour']        = dfCACrash['4DigTime'].str[:2]
dfCACrash['Min']         = dfCACrash['4DigTime'].str[2:]
dfCACrash['Time']        = dfCACrash['Hour']+":"+dfCACrash['Min']
dfCACrash['Data_Source'] = "CHP/SWITRS"

#Convert severity
dfCACrash.loc[dfCACrash['COLLISION_SEVERITY'].isin([1]),  'Crash_Severity'] = 'Fatal'
dfCACrash.loc[dfCACrash['COLLISION_SEVERITY'].isin([2]),  'Crash_Severity'] = 'Severe injury'
dfCACrash.loc[dfCACrash['COLLISION_SEVERITY'].isin([3]),  'Crash_Severity'] = 'Other visible injury'
dfCACrash.loc[dfCACrash['COLLISION_SEVERITY'].isin([4]),  'Crash_Severity'] = 'Complaint of pain'
dfCACrash.loc[dfCACrash['COLLISION_SEVERITY'].isin([0]),  'Crash_Severity'] = 'Property damage only'
#Convert crash type
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["A"]),  'Crash_Type'] = 'Head-on'
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["B"]),  'Crash_Type'] = 'Sideswipe'
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["C"]),  'Crash_Type'] = 'Rear end'
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["D"]),  'Crash_Type'] = 'Angle-broadside'
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["E"]),  'Crash_Type'] = 'Hit object'
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["F"]),  'Crash_Type'] = 'Overturned'
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["G"]),  'Crash_Type'] = 'Vehicle/pedestrian'
dfCACrash.loc[dfCACrash['TYPE_OF_COLLISION'].isin(["H"]),  'Crash_Type'] = 'Other'
#convert lighting
dfCACrash.loc[dfCACrash['LIGHTING'].isin(["A"]),  'Lighting'] = "Daylight"
dfCACrash.loc[dfCACrash['LIGHTING'].isin(["B"]),  'Lighting'] = "Dusk - Dawn"
dfCACrash.loc[dfCACrash['LIGHTING'].isin(["C"]),  'Lighting'] = "Dark - Street Lights"
dfCACrash.loc[dfCACrash['LIGHTING'].isin(["D"]),  'Lighting'] = "Dark - No Street Lights"
dfCACrash.loc[dfCACrash['LIGHTING'].isin(["E"]),  'Lighting'] = "Dark - Street Lights Not Functioning"
dfCACrash.loc[dfCACrash['LIGHTING'].isin(["-"]),  'Lighting'] = "Not Stated"
#Convert weather 1
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["A"]),  'Weather_1'] = "Clear"
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["B"]),  'Weather_1'] = "Cloudy"
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["C"]),  'Weather_1'] = "Raining"
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["D"]),  'Weather_1'] = "Snowing"
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["E"]),  'Weather_1'] = "Fog"
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["F"]),  'Weather_1'] = "Other"
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["G"]),  'Weather_1'] = "Windy"
dfCACrash.loc[dfCACrash['WEATHER_1'].isin(["-"]),  'Weather_1'] = "Not Stated"
# convert weather 2
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["A"]),  'Weather_2'] = "Clear"
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["B"]),  'Weather_2'] = "Cloudy"
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["C"]),  'Weather_2'] = "Raining"
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["D"]),  'Weather_2'] = "Snowing"
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["E"]),  'Weather_2'] = "Fog"
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["F"]),  'Weather_2'] = "Other"
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["G"]),  'Weather_2'] = "Windy"
dfCACrash.loc[dfCACrash['WEATHER_2'].isin(["-"]),  'Weather_2'] = "Not Stated"
#Convert violation
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["0"]),  'Violation'] = "Unknown"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["1"]),  'Violation'] = "Driving/Biking Under the Influence"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["2"]),  'Violation'] = "Impeding Traffic"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["3"]),  'Violation'] = "Unsafe Speed"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["4"]),  'Violation'] = "Following Too Closely"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["5"]),  'Violation'] = "Wrong Side of Road"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["6"]),  'Violation'] = "Improper Passing"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["7"]),  'Violation'] = "Unsafe Lane Change"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["8"]),  'Violation'] = "Improper Turning"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["9"]),  'Violation'] = "Auotomobile Right of Way"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["10"]),  'Violation'] = "Pedestrian Right of Way"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["11"]),  'Violation'] = "Pedestrian Violation"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["12"]),  'Violation'] = "Traffic Signals and Signs"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["13"]),  'Violation'] = "Hazardous Parking"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["17"]),  'Violation'] = "Other Hazardous Violation"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["18"]),  'Violation'] = "Other Than Driver or Pedestrian"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["21"]),  'Violation'] = "Unsafe Backing or Starting"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["22"]),  'Violation'] = "Other Improper Driving"
dfCACrash.loc[dfCACrash['PCF_VIOL_CATEGORY'].isin(["-"]),  'Violation'] = "Not Stated"
#Convert road surface
dfCACrash.loc[dfCACrash['ROAD_SURFACE'].isin(["A"]),  'Road_Surface'] = "Dry"
dfCACrash.loc[dfCACrash['ROAD_SURFACE'].isin(["B"]),  'Road_Surface'] = "Wet"
dfCACrash.loc[dfCACrash['ROAD_SURFACE'].isin(["C"]),  'Road_Surface'] = "Snowy or Icy"
dfCACrash.loc[dfCACrash['ROAD_SURFACE'].isin(["D"]),  'Road_Surface'] = "Slippery (Muddy, Oily, etc.)"
dfCACrash.loc[dfCACrash['ROAD_SURFACE'].isin(["-"]),  'Road_Surface'] = "Not Stated"
#Convert road condition 1
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["A"]),  'Road_Condition_1'] = "Holes, Deep Ruts"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["B"]),  'Road_Condition_1'] = "Loose Material on Roadway"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["C"]),  'Road_Condition_1'] = "Obstruction on Roadway"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["D"]),  'Road_Condition_1'] = "Construction or Repair Zone"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["E"]),  'Road_Condition_1'] = "Reduced Roadway Width"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["F"]),  'Road_Condition_1'] = "Flooded"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["G"]),  'Road_Condition_1'] = "Other"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["H"]),  'Road_Condition_1'] = "No Unusual Condition"
dfCACrash.loc[dfCACrash['ROAD_COND_1'].isin(["-"]),  'Road_Condition_1'] = "Not Stated"
# conver road condition 2
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["A"]),  'Road_Condition_2'] = "Holes, Deep Ruts"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["B"]),  'Road_Condition_2'] = "Loose Material on Roadway"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["C"]),  'Road_Condition_2'] = "Obstruction on Roadway"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["D"]),  'Road_Condition_2'] = "Construction or Repair Zone"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["E"]),  'Road_Condition_2'] = "Reduced Roadway Width"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["F"]),  'Road_Condition_2'] = "Flooded"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["G"]),  'Road_Condition_2'] = "Other"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["H"]),  'Road_Condition_2'] = "No Unusual Condition"
dfCACrash.loc[dfCACrash['ROAD_COND_2'].isin(["-"]),  'Road_Condition_2'] = "Not Stated"
#Convert pedestrian action
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["A"]),  'Pedestrian_Action'] = "No Pedestrian Involved"
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["B"]),  'Pedestrian_Action'] = "Crossing in Crosswalk at Intersection"
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["C"]),  'Pedestrian_Action'] = "Crossing in Crosswalk Not at Intersection"
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["D"]),  'Pedestrian_Action'] = "Crossing Not in Crosswalk"
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["E"]),  'Pedestrian_Action'] = "In Road, Including Shoulder"
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["F"]),  'Pedestrian_Action'] = "Not in Road"
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["G"]),  'Pedestrian_Action'] = "Approaching/Leaving School Bus"
dfCACrash.loc[dfCACrash['PED_ACTION'].isin(["-"]),  'Pedestrian_Action'] = "Not Stated"
#Convert hit and run
dfCACrash.loc[dfCACrash['HIT_AND_RUN'].isin(["F"]), 'Hit_and_Run'] = "Felony"
dfCACrash.loc[dfCACrash['HIT_AND_RUN'].isin(["M"]), 'Hit_and_Run'] = "Misdemeanor"
dfCACrash.loc[dfCACrash['HIT_AND_RUN'].isin(["N"]), 'Hit_and_Run'] = "Not Hit and Run"
#Convert MVIW
dfCACrash.loc[dfCACrash['MVIW'].isin(["A"]), 'Motor_Vehicle_Interacted_With'] = "Non-Collision"
dfCACrash.loc[dfCACrash['MVIW'].isin(["B"]), 'Motor_Vehicle_Interacted_With'] = "Pedestrian"
dfCACrash.loc[dfCACrash['MVIW'].isin(["C"]), 'Motor_Vehicle_Interacted_With'] = "Other Motor Vehicle"
dfCACrash.loc[dfCACrash['MVIW'].isin(["D"]), 'Motor_Vehicle_Interacted_With'] = "Motor Vehicle on Other Roadway"
dfCACrash.loc[dfCACrash['MVIW'].isin(["E"]), 'Motor_Vehicle_Interacted_With'] = "Parked Motor Vehicle"
dfCACrash.loc[dfCACrash['MVIW'].isin(["F"]), 'Motor_Vehicle_Interacted_With'] = "Train"
dfCACrash.loc[dfCACrash['MVIW'].isin(["G"]), 'Motor_Vehicle_Interacted_With'] = "Bicycle"
dfCACrash.loc[dfCACrash['MVIW'].isin(["H"]), 'Motor_Vehicle_Interacted_With'] = "Animal"
dfCACrash.loc[dfCACrash['MVIW'].isin(["I"]), 'Motor_Vehicle_Interacted_With'] = "Fixed Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["J"]), 'Motor_Vehicle_Interacted_With'] = "Other Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["0"]), 'Motor_Vehicle_Interacted_With'] = "Non-Collision and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["1"]), 'Motor_Vehicle_Interacted_With'] = "Pedestrian and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["2"]), 'Motor_Vehicle_Interacted_With'] = "Other Motor Vehicle and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["3"]), 'Motor_Vehicle_Interacted_With'] = "Motor Vehicle on Other Roadway and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["4"]), 'Motor_Vehicle_Interacted_With'] = "Parked Motor Vehicle and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["5"]), 'Motor_Vehicle_Interacted_With'] = "Train and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["6"]), 'Motor_Vehicle_Interacted_With'] = "Bicycle and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["7"]), 'Motor_Vehicle_Interacted_With'] = "Animal and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["8"]), 'Motor_Vehicle_Interacted_With'] = "Fixed Object and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["9"]), 'Motor_Vehicle_Interacted_With'] = "Other Object and Additional Object"
dfCACrash.loc[dfCACrash['MVIW'].isin(["-"]), 'Motor_Vehicle_Interacted_With'] = "Not Stated"
#convert case ID
dfCACrash['CA_Case_ID']            = dfCACrash['CASE_ID']
dfCACrash['NV_Accident_Num']       = np.nan
dfCACrash['NV_Accident_Rec_Num']   = np.nan
#Convert number injured/killed
dfCACrash['Num_Killed']            = dfCACrash['NUMBER_KILLED']
dfCACrash['Num_Injured']           = dfCACrash['NUMBER_INJURED']
dfCACrash['Num_Ped_Killed']        = dfCACrash['COUNT_PED_KILLED']
dfCACrash['Num_Ped_Injured']       = dfCACrash['COUNT_PED_INJURED']
dfCACrash['Num_Bicyclist_Killed']  = dfCACrash['COUNT_BICYCLIST_KILLED']
dfCACrash['Num_Bicyclist_Injured'] = dfCACrash['COUNT_BICYCLIST_INJURED']
#Convert number vehicles/parties
dfCACrash['Num_Vehicles']          = np.nan
dfCACrash['Num_Parties']           = dfCACrash['PARTY_COUNT']
#Convert alcohol/bike/ped involvement
dfCACrash['Alcohol_Involved']      = dfCACrash['ALCOHOL_INVOLVED']
dfCACrash['Pedestrian_Involved']   = dfCACrash['PEDESTRIAN_ACCIDENT']
dfCACrash['Bicycle_Involved']      = dfCACrash['BICYCLE_ACCIDENT']
dfCACrash['Corridor_ID']           = np.nan

# final list of fields
dfCACrash = dfCACrash[['State',
           'CA_Case_ID',
           'NV_Accident_Num',
           'NV_Accident_Rec_Num',
           'Corridor_ID',
           'County',
           'City',
           'Year',
           'Date',
           'Time',
           'Weather_1',
           'Weather_2',
           'Crash_Severity',
           'Num_Killed',
           'Num_Injured',
           'Num_Ped_Killed',
           'Num_Ped_Injured',
           'Num_Bicyclist_Killed',
           'Num_Bicyclist_Injured',
           'Crash_Type',
           'Num_Vehicles',
           'Num_Parties',
           'Violation',
           'Hit_and_Run',
           'Motor_Vehicle_Interacted_With',
           'Pedestrian_Action', 
           'Road_Condition_1',
           'Road_Condition_2',
           'Road_Surface',
           'Lighting',
           'Pedestrian_Involved',
           'Bicycle_Involved',
           'Alcohol_Involved',
           'Data_Source',
           'POINT_X',
           'POINT_Y']].copy()

## NV data transformation

In [ ]:

## NV Data Transformation
# set fields for time and case info
dfNVCrash['CA_Case_ID']           = np.nan
dfNVCrash['NV_Accident_Num']      = dfNVCrash['NV Accident Num']
dfNVCrash['NV_Accident_Rec_Num']  = dfNVCrash['NV Accident Rec Num']
dfNVCrash['City']                 = np.nan
dfNVCrash['Year']                 = dfNVCrash['Collision_Year']
dfNVCrash['Date']                 = dfNVCrash['Collision_Date']
dfNVCrash['Time']                 = dfNVCrash['Collision_Time']
dfNVCrash['Num_Vehicles']         = dfNVCrash['Total Vehicles']
dfNVCrash['Num_Parties']          = np.nan
dfNVCrash['Data_Source']          = "NDOT"

# Convert NV crash type and severity
dfNVCrash['Crash_Severity']       = dfNVCrash['COLLISION_SEVERITY']
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["ANGLE"]),  'Crash_Type']         = 'Angle-broadside'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["BACKING"]),  'Crash_Type']       = 'Backing'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["HEAD-ON"]),  'Crash_Type']       = 'Head-on'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["NON-COLLISION"]),  'Crash_Type'] = 'Non-collision'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["REAR-END"]),  'Crash_Type']      = 'Rear end'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["REAR-TO-REAR"]),  'Crash_Type']  = 'Other'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["SIDESWIPE, MEETING"]),  'Crash_Type'] = 'Sideswipe'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["SIDESWIPE, OVERTAKING"]),  'Crash_Type'] = 'Sideswipe'
dfNVCrash.loc[dfNVCrash['Crash Type'].isin(["UNKNOWN"]),  'Crash_Type'] = 'Unknown'
#Convert # injured/killed
dfNVCrash['Num_Killed']   = dfNVCrash['Fatalities']
dfNVCrash['Num_Killed'].fillna(0)
dfNVCrash['Num_Injured'] = dfNVCrash['Injured']
dfNVCrash['Num_Injured'].fillna(0)
dfNVCrash['Num_Ped_Killed'] = np.nan
dfNVCrash['Num_Ped_Injured'] = np.nan
dfNVCrash['Num_Bicyclist_Killed'] = np.nan
dfNVCrash['Num_Bicyclist_Injured'] = np.nan
# convert crash info
dfNVCrash['Violation']  = "N/A"
dfNVCrash['Hit_and_Run'] = "N/A"
dfNVCrash['Motor_Vehicle_Interacted_With'] = "N/A"
dfNVCrash['Pedestrian_Action'] = "N/A"
# Process lighting
dfNVCrash['Lighting'] = dfNVCrash['LIGHTING']
dfNVCrash.loc[dfNVCrash['Lighting'].isin(["Dusk-Dawn"]),  'Lighting'] = "Dusk - Dawn"
dfNVCrash.loc[dfNVCrash['Lighting'].isin(["Dusk-dawn"]),  'Lighting'] = "Dusk - Dawn"
dfNVCrash.loc[dfNVCrash['Lighting'].isin(["Dark - street lights"]),  'Lighting'] = "Dark - Street Lights"
dfNVCrash.loc[dfNVCrash['Lighting'].isin(["Dark - no street lights"]),  'Lighting'] = "Dark - No Street Lights"
dfNVCrash.loc[dfNVCrash['Lighting'].isin(["Dark - Unknown Lighting"]),  'Lighting'] = "Dark - Unknown Lighting"
dfNVCrash.loc[dfNVCrash['Lighting'].isin(["Not stated"]),  'Lighting'] = "Not Stated"
#Process alcohol involvement (check outputs)
dfNVCrash['V1 Driver Factors'] = dfNVCrash['V1 Driver Factors'].fillna("Not stated")
dfNVCrash['V2 Driver Factors'] = dfNVCrash['V2 Driver Factors'].fillna("Not stated")
dfNVCrash.loc[dfNVCrash['V1 Driver Factors'].str.contains("DRINKING"), 'Alcohol_Involved'] = "Y"
dfNVCrash.loc[dfNVCrash['V2 Driver Factors'].str.contains("DRINKING"), 'Alcohol_Involved'] = "Y"
#Process bike/ped involvement (check outputs)
dfNVCrash['V1 All Events'] = dfNVCrash['V1 All Events'].fillna("Not stated")
dfNVCrash['V2 All Events'] = dfNVCrash['V2 All Events'].fillna("Not stated")
dfNVCrash.loc[dfNVCrash['V1 All Events'].str.contains("PEDESTRIAN"), 'Pedestrian_Involved'] = "Y"
dfNVCrash.loc[dfNVCrash['V2 All Events'].str.contains("PEDESTRIAN"), 'Pedestrian_Involved'] = "Y"
dfNVCrash.loc[dfNVCrash['V1 All Events'].str.contains("PEDAL CYCLE"), 'Bicycle_Involved'] = "Y"
dfNVCrash.loc[dfNVCrash['V2 All Events'].str.contains("PEDAL CYCLE"), 'Bicycle_Involved'] = "Y"
# #Convert road surface
dfNVCrash['Factors Roadway'] = dfNVCrash['Factors Roadway'].fillna("Not stated")
dfNVCrash['HWY Factors'] = dfNVCrash['HWY Factors'].fillna("Not stated")
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("DRY"), 'Road_Surface'] = "Dry"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("WET"), 'Road_Surface'] = "Wet"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("WATER"), 'Road_Surface'] = "Wet"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("ICE"), 'Road_Surface'] = "Snowy or Icy"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("SNOW"), 'Road_Surface'] = "Snowy or Icy"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("SLUSH"), 'Road_Surface'] = "Snowy or Icy"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("OTHER"), 'Road_Surface'] = "Other"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("NA"), 'Road_Surface'] = "Not stated"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("UNKNOWN"), 'Road_Surface'] = "Unknown"
dfNVCrash.loc[dfNVCrash['Factors Roadway'].str.contains("Not stated"), 'Road_Surface'] = "Not stated"#Convert road condition
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("NONE"), 'Road_Condition_1'] = "No Unusual Condition"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("UNKNOWN"), 'Road_Condition_1'] = "Unknown"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("WET, ICY, SNOW, SLUSH"), 'Road_Condition_1'] = "Weather"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("WEATHER"), 'Road_Condition_1'] = "Weather"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("BACKUP"), 'Road_Condition_1'] = "Other"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("GLARE"), 'Road_Condition_1'] = "Other"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("OTHER"), 'Road_Condition_1'] = "Other"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("VISUAL OBSTRUCTION"), 'Road_Condition_1'] = "Other"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("ROAD OBSTRUCTION"), 'Road_Condition_1'] = "Obstruction on Roadway"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("DEBRIS"), 'Road_Condition_1'] = "Loose Material on Roadway"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("WORK ZONE"), 'Road_Condition_1'] = "Construction or Repair Zone"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("ANIMAL"), 'Road_Condition_1'] = "Wildlife"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("RUTS"), 'Road_Condition_1'] = "Holes, Deep Ruts"
dfNVCrash.loc[dfNVCrash['HWY Factors'].str.contains("Not stated"), 'Road_Condition_1'] = "Not Stated"
dfNVCrash['Road_Condition_2'] = np.nan

#Rename to match CA data
dfNVCrash['Weather_1']   = dfNVCrash['Weather']
dfNVCrash['Weather_2']   = np.nan
dfNVCrash['POINT_X']     = dfNVCrash['X']
dfNVCrash['POINT_Y']     = dfNVCrash['Y']
dfNVCrash['Corridor_ID'] = np.nan

# final list of fields
dfNVCrash = dfNVCrash[['State',
           'CA_Case_ID',
           'NV_Accident_Num',
           'NV_Accident_Rec_Num',
           'Corridor_ID',
           'County',
           'City',
           'Year',
           'Date',
           'Time',
           'Weather_1',
           'Weather_2',
           'Crash_Severity',
           'Num_Killed',
           'Num_Injured',
           'Num_Ped_Killed',
           'Num_Ped_Injured',
           'Num_Bicyclist_Killed',
           'Num_Bicyclist_Injured',
           'Crash_Type',
           'Num_Vehicles',
           'Num_Parties',
           'Violation',
           'Hit_and_Run',
           'Motor_Vehicle_Interacted_With',
           'Pedestrian_Action', 
           'Road_Condition_1',
           'Road_Condition_2',
           'Road_Surface',
           'Lighting',
           'Pedestrian_Involved',
           'Bicycle_Involved',
           'Alcohol_Involved',
           'Data_Source',
           'POINT_X',
           'POINT_Y']].copy()

## NV 2021 Data

In [14]:

from arcgis import GeoAccessor

arcpy.env.overwriteOutput = True
arcpy.env.workspace = "F:/GIS/PROJECTS/ResearchAnalysis/Monitoring/Data/Crash/2021 Crash Data/NV/Statewide Data 2021.gdb"

crash_featureclass = "Crash_Data_2021"
person_table = "Person_Table_2021"
vehicle_table = "Vehicle_Table_2021"

def import_table_from_fgb(tablename):
    data = []

    # Use SearchCursor to iterate through the feature class
    fields = [field.name for field in arcpy.ListFields(tablename)]
    with arcpy.da.SearchCursor(tablename, fields) as cursor:
        for row in cursor:
            data.append(row)
    # Convert the list of tuples to a Pandas DataFrame
    df = pd.DataFrame(data, columns=fields)
    return df
def renamecolumns(df, column_mapping):
    df = df.rename(columns=column_mapping).drop(columns=[col for col in df.columns if col not in column_mapping])
    return df


person_df = import_table_from_fgb(person_table)
vehicle_df = import_table_from_fgb(vehicle_table)

crash_sdf = GeoAccessor.from_featureclass(crash_featureclass)



## Map and rename columns for dfs from a dictionary

In [ ]:
crash_column_mapping = {
    'CrashNum':'NV_Accident_Num',
    'aNo':'NV_Accident_Rec_Num',
    'CountyLocated':'County',
    'CrashDate':'Date',
    'CrashTime':'Time',
    

}

In [ ]:


# Update the 'Category' column based on whether any dictionary key is contained in the 'ID' value
df['Category'] = df['ID'].apply(lambda x: next((v for k, v in lookup_dict.items() if k in x), x))